In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn import tree


In [2]:
data = pd.read_csv('housing.csv')

In [3]:
data.head()

,0.00632,18,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24
0,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
1,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7


In [4]:
data.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'PRICE']

In [5]:
data.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
count,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000
mean,3.620667,11.350495,11.154257,0.069307,0.554728,6.284059,68.581584,3.794459,9.566337,408.459406,18.461782,356.594376,12.668257,22.529901
std,8.608572,23.343704,6.855868,0.254227,0.115990,0.703195,28.176371,2.107757,8.707553,168.629992,2.162520,91.367787,7.139950,9.205991
min,0.009060,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082210,0.000000,5.190000,0.000000,0.449000,5.885000,45.000000,2.100000,4.000000,279.000000,17.400000,375.330000,7.010000,17.000000
50%,0.259150,0.000000,9.690000,0.000000,0.538000,6.208000,77.700000,3.199200,5.000000,330.000000,19.100000,391.430000,11.380000,21.200000
75%,3.678220,12.500000,18.100000,0.000000,0.624000,6.625000,94.100000,5.211900,24.000000,666.000000,20.200000,396.210000,16.960000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [6]:
corr = data.corr()
corr.shape

(14, 14)

In [7]:
corr['PRICE'].sort_values(ascending = False)

PRICE      1.000000
RM         0.695365
ZN         0.360393
B          0.333394
DIS        0.249896
CHAS       0.175364
AGE       -0.376932
RAD       -0.381690
CRIM      -0.388249
NOX       -0.427295
TAX       -0.468543
INDUS     -0.484126
PTRATIO   -0.508411
LSTAT     -0.738187
Name: PRICE, dtype: float64

In [8]:
data = data.drop(['B','DIS','ZN'],axis = 1)

In [9]:
data['CHAS'].value_counts()

CHAS
0    470
1     35
Name: count, dtype: int64

In [10]:
# plt.figure(figsize=(20,20))
# sns.heatmap(corr, cbar=True, square= True, fmt='.1f', annot=True, annot_kws={'size':15}, cmap='Greens')

In [11]:
x = data.drop(['PRICE'], axis = 1)
y = data['PRICE'].copy()

In [12]:
#x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 42)

In [13]:
# x['CHAS'].value_counts()

In [14]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_index, test_index in split.split(data, data[['CHAS']]):
    x_train, x_test = data.iloc[train_index], data.iloc[test_index]
    y_train, y_test = data.iloc[train_index], data.iloc[test_index]


In [15]:
x_train = x_train.drop(['PRICE'],axis = 1)
x_test = x_test.drop(['PRICE'],axis = 1)
y_train = y_train['PRICE'].copy()
y_test = y_test['PRICE'].copy()

In [16]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 353 entries, 122 to 455
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     353 non-null    float64
 1   INDUS    353 non-null    float64
 2   CHAS     353 non-null    int64  
 3   NOX      353 non-null    float64
 4   RM       353 non-null    float64
 5   AGE      353 non-null    float64
 6   RAD      353 non-null    int64  
 7   TAX      353 non-null    int64  
 8   PTRATIO  353 non-null    float64
 9   LSTAT    353 non-null    float64
dtypes: float64(7), int64(3)
memory usage: 30.3 KB


In [21]:
model = LinearRegression()
#model = RandomForestRegressor()
#model = XGBRegressor()
#model = tree.DecisionTreeRegressor()
model.fit(x_train,y_train)
predict = model.predict(x_train)
mse = mean_squared_error(y_train,predict)
rmse = np.sqrt(mse)
r2 = r2_score(y_train, predict)
print(r2)

0.6923375827135256


In [22]:
rmse

5.1074038239273385

In [23]:
predict = model.predict(x_test)
mse = mean_squared_error(y_test,predict)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predict)
print(r2)

0.7183102865029047


In [24]:
rmse

4.8623648647307585